In [8]:
import torch.optim as optim
from torch_geometric.data import Data
from sklearn.metrics import f1_score
import json
import functools
import numpy as np
import typing

In [9]:
EVAL_METRICS = ['mic_f1', 'mac_f1']
DEFAULT_CMP_BY = 'mic_f1'

@functools.total_ordering
class EvalResults(dict):
    def __init__(self, mic_f1, mac_f1, cmp_by=DEFAULT_CMP_BY):
        super().__init__({
            'mic_f1': mic_f1,
            'mac_f1': mac_f1
        })
        self.cmp_by = cmp_by

    def __lt__(self, other):
        if not isinstance(other, EvalResults):
            return NotImplemented
        fst = self.cmp_by
        snd = [m for m in EVAL_METRICS if m != self.cmp_by][0]
        if self[fst] < other[fst]:
            return True
        elif self[fst] > other[fst]:
            return False
        # self[fst] = other[fst]
        elif self[snd] < other[snd]:
            return True
        else:
            return False 

    def __eq__(self, other):
        if not isinstance(other, EvalResults):
            return NotImplemented
        # Equal if all metrics are the same
        return all(self[m] == other[m] for m in EVAL_METRICS)

    def __le__(self, other):
        return self < other or self == other

    def __repr__(self):
        return (f"mic_f1: {self['mic_f1']:.4f}, "
                f"mac_f1: {self['mac_f1']:.4f}")

    def to_dict(self):
        return { 'mic_f1': self['mic_f1'], 'mac_f1': self['mac_f1'] }

    def to_serialisable(self):
        return { k:str(v) for k,v in self.to_dict().items() }

    @staticmethod
    def average(results, std=True):
        mic_f1s = [res['mic_f1'] for res in results]
        mac_f1s = [res['mac_f1'] for res in results]

        avgs = EvalResults(np.mean(mic_f1s), np.mean(mac_f1s))
        stds = None if not std else EvalResults(np.std(mic_f1s), np.std(mac_f1s))
        
        return avgs, stds

In [10]:
def evaluate(model, data, mask, cmp_by=DEFAULT_CMP_BY):
    """
    Evaluates model and returns its validation accuracy, 
    micro-F1 and macro-F1 scores on given mask.
    """
    model.eval()
    with torch.no_grad():  # disable gradient computation during evaluation
        # forward pass
        out = model(data.x, data.edge_index)
        # predict the class with max score
        pred = out.argmax(dim=1)
        true_labels = data.y[mask]
        # calculate F1 scores (`f1_score` expects the inputs to be on the CPU)
        mic_f1 = f1_score(true_labels.cpu(), pred[mask].cpu(), average='micro') # equivalent to accuracy for this task
        mac_f1 = f1_score(true_labels.cpu(), pred[mask].cpu(), average='macro')

    return EvalResults(mic_f1, mac_f1, cmp_by=cmp_by)

In [11]:
def init_training(params):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    data, dataset = load_data(params['dataset'], data_only=False)
    params["n_classes"] = dataset.num_classes  # number of target classes
    params["input_dim"] = dataset.num_features  # size of input features
    
    model = set_model(params, device)
    model.param_init()
    
    optimiser = optim.Adam(model.parameters(), 
                           lr=params['lr'], 
                           weight_decay=params['weight_decay'])
    loss_fn = nn.CrossEntropyLoss()
    
    return model, data, optimiser, loss_fn

In [12]:
def train_only(params: typing.Dict,
               cmp_by=DEFAULT_CMP_BY,
               report_per_period=1000,
               print_results=True):
    """
    Trains a node classification model and
    returns the trained model object.
    """
    model, data, optimiser, loss_fn = init_training(params)
    n_epochs = params['epochs']

    # variables for early stopping
    best_results = EvalResults(-1,-1)  # best validation results
    prev_loss = float('inf')
    consec_worse_epochs = 0  # number of consecutive epochs with degrading results
    # k: stop if epochs_dec_acc >= patience
    patience = params['max_patience']

    # standard training with backpropagation
    for epoch in range(n_epochs):
        model.train()
        optimiser.zero_grad()
        out = model(data.x, data.edge_index) # forward pass
        loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
        loss.backward() # backward pass
        optimiser.step()

        # evaluate on validation set
        results = evaluate(model, data, data.val_mask, cmp_by=cmp_by)

        # early stopping
        if results >= best_results:
            best_results = results
            consec_worse_epochs = 0
        else:
            consec_worse_epochs += 1

        # patience exceeded -> stop training
        if consec_worse_epochs >= patience:
            if print_results:
                print(f"Early stopping at epoch {epoch+1}")
                print(f"Best results: {best_results}")
            break

        # print training progress
        if (epoch+1) % report_per_period == 0:
            print(f"Epoch {epoch + 1}/{n_epochs}...")
            print(f"Loss: {loss};")
            print(f"Validation Results:\n{results}\n")

    return model, best_results

In [13]:
def train_and_tune(params, hyperparam, hyperparam_values, 
                   cmp_by=DEFAULT_CMP_BY,
                   report_per_period=1000,
                   print_results=True):
    """
    Trains the model and performs hyperparameter tuning.

    Args:
    - params: Training parameters.
    - hyperparam: Name of the hyperparameter to tune (if any).
    - hyperparam_values: Values to test for the hyperparameter (if any).
    - report_per_period: Frequency of training status reports.
    - metric: Metric to optimise during training; one of ["accuracy", "micro_f1", or "macro_f1"].

    Returns:
    - if hyperparam_values = None, a pair: (trained model, its training performance) - same as [train]
    - otherwise, a triple: (optimal trained model, ts training performance, its hyperparameter value)
    """
    best_hyperparam_val, best_model = None, None, 
    best_results = EvalResults(-1,-1)

    for val in hyperparam_values:
        params[hyperparam] = val
        model, results = train_only(params, cmp_by, 
                                    report_per_period, 
                                    print_results)
        if results > best_results:
            best_results = results
            best_hyperparam_val = val
            best_model = model

    return best_model, best_results, best_hyperparam_val

In [14]:
def train(params, hyperparam=None, 
          hyperparam_values=None, 
          cmp_by=DEFAULT_CMP_BY,
          report_per_period=1000,
          print_results=True):
    """
    Wrapper training function.
    Returns a triple: (model, training results, training hyperparameters)
    """
    model_name = params['model_name']
    if model_name not in MODEL_SPEC_HYPERPARAM: # train without tuning
        model, res = train_only(params, cmp_by, report_per_period, print_results)
        return model, res, params
    else: 
        model, res, hyperparam_val = train_and_tune(params, hyperparam, 
                                                    hyperparam_values, 
                                                    cmp_by, report_per_period, 
                                                    print_results)
        tuned_hyperparam = MODEL_SPEC_HYPERPARAM[model_name]
        params[tuned_hyperparam] = hyperparam_val
        return model, res, params

In [19]:
TRAIN_LAYERS = range(2,21,2)

In [22]:
def train_diff_layers_model(params,
                            layers=TRAIN_LAYERS,
                            cmp_by=DEFAULT_CMP_BY,
                            report_per_period=100,
                            print_results=True):
    model_name = params['model_name']
    hyperparam = MODEL_SPEC_HYPERPARAM.get(model_name, None)
    hyperparam_values = MODEL_HYPERPARAM_RANGE.get(model_name, None)
    
    layers_to_model = dict()
    layers_to_hyperparams = dict()
    for n in layers:
        curr_params = params.copy()
        curr_params['n_layers'] = n
        model, res, hyperparams = train(curr_params, hyperparam, hyperparam_values, 
                                        cmp_by, report_per_period, print_results)
        layers_to_model[n] =  model
        layers_to_hyperparams[n] = hyperparams

    return layers_to_model, layers_to_hyperparams

## Testing

In [23]:
def test(model, dataset_name):
    data = load_data(dataset_name, data_only=True)
    return evaluate(model, data, data.test_mask)

In [24]:
def train_and_test(params,
                   layers=TRAIN_LAYERS,
                   cmp_by=DEFAULT_CMP_BY,
                   report_per_period=1000,
                   print_results=True,
                   export_results=True):
    dataset_name = params['dataset']
    model_name = params['model_name']
    layers_to_model, layers_to_hyperparams = train_diff_layers_model(params, 
                                                                     layers, 
                                                                     cmp_by,
                                                                     report_per_period,
                                                                     print_results)
    layers_to_results = dict()  # number of layers : test results
    for n, model in layers_to_model.items():
        layers_to_results[n] = test(model, dataset_name)
        layers_to_model[n] = model
        if export_results:
            torch.save(model.state_dict(), f'./results/{dataset_name}/{n}_layers.pt')

    if print_results:
        print(f"Test results for {model_name} on {dataset_name}:")
        for n, results in layers_to_results.items():
            print(f"{n}-layer model:")
            print(f"Hyperparameter setting:")
            print(layers_to_hyperparams[n])
            print(results)
        print()
    if export_results:
        with open(f'./results/{dataset_name}/results.json', 'w') as fp:
            data = {n : metrics.to_serialisable() for n,metrics in layers_to_results.items() }
            json.dump(data, fp, indent=4)

    return layers_to_model, layers_to_results, layers_to_hyperparams

In [25]:
def stderr(xs):
    return np.std(xs, ddof=1) / np.sqrt(len(xs))

In [26]:
DEFAULT_TRAINING_PARAMS = {
    "lr": 0.01,  # learning rate
    "weight_decay": 0.0005,  # weight_decay
    "epochs": 400,  # number of total training epochs
    "max_patience": 5, # number of k for early stopping
    "hid_dim": 64, # hidden dimensions
    "init_res_weight": 0
}

def training_params(model_name, dataset_name, init_res_weight=0, n_layers=2):
    params = DEFAULT_TRAINING_PARAMS.copy()
    params['model_name'] = model_name
    params['dataset'] = dataset_name
    params['init_res_weight'] = init_res_weight
    params['n_layers'] = n_layers
    return params